# Trace collection

In [ ]:
import time
import holoviews as hv
import numpy as np
from hashlib import sha1
from tqdm.notebook import tqdm
from pyecsca.sca.trace_set import PickleTraceSet
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces

from client import DeviceTarget

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
hv.extension("bokeh")

Setup the target, runs at frequency of 15MHz. This is also set in the code via the `F_CPU` define in `hal/Makefile.hal`.

In [ ]:
target = DeviceTarget()

target.timeout = 2000
base_freq = 15000000
target.scope.io.clkout = base_freq
target.scope.adc.clk_freq = base_freq
target.scope.adc.samples = 50000

Flash the target with the firmware, takes about 50 seconds.
Only needed if the code changed.
Need to run `make` first.

In [ ]:
target.flash("../micro-ecc-CWNANO.hex")

Connect to the target.

In [ ]:
target.connect()

Initialize the PRNG on the target.

In [ ]:
seed = bytes.fromhex("cafebabe")
target.init_prng(seed)

Generate a keypair on the target.

In [ ]:
target.generate_keypair()

Export the public key.

In [ ]:
pubkey = target.export()
print(pubkey)

Pick and hash a message.

In [ ]:
msg = b"This is the message"
hash = sha1(msg).digest()

Collect the traces, storing the signatures and duration for each trace.

In [ ]:
traces = []
for i in tqdm(range(10)):
    target.scope.arm()
    start = time.perf_counter()
    
    signature = target.sign(hash)
    
    end = time.perf_counter()
    duration = end - start
    
    target.scope.capture()
    
    meta = {
        "signature": signature,
        "duration": duration
    }
    trace = Trace(samples=target.scope.get_last_trace(), meta=meta)
    traces.append(trace)
trace_set = PickleTraceSet(*traces, pubkey=pubkey, hash=hash, msg=msg)

Stop the target and disconnect from it. If you want to connect to it again you need to run `target.reset()` and `target.connect()`.

In [ ]:
target.halt()
target.disconnect()

Plot a couple of traces.

In [ ]:
plot_traces(*traces[:2]).opts(width=950, height=600)

Save the trace set.

In [ ]:
trace_set.write("traces.pickle")

Verify the signatures.

In [ ]:
from client import verify_signature

In [ ]:
for trace in trace_set:
    print(verify_signature(trace_set.pubkey, trace.meta["signature"], hash))